## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [1]:
import pandas as pd

In [2]:
!pip install gdown

In [3]:
!gdown 1lEpoRKczv5EvZhff9O2I-0JkdHnbe_Mq -O ./data/transactions.csv

Downloading...
From: https://drive.google.com/uc?id=1lEpoRKczv5EvZhff9O2I-0JkdHnbe_Mq
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/transactions.csv
100%|████████████████████████████████████████| 314M/314M [01:36<00:00, 3.25MB/s]


In [4]:
transactions = pd.read_csv('./data/transactions.csv', nrows=1000000)
transactions.columns

Index(['customer_id', 'tr_datetime', 'mcc_code', 'tr_type', 'amount',
       'term_id'],
      dtype='object')

In [5]:
tr_types_samples = transactions.tr_type.sample(1000)
tr_types_samples.head()

520249    2010
2873      1030
220057    1010
619527    1010
517731    2010
Name: tr_type, dtype: int64

In [6]:
!gdown 1EP3KrATWS1I_qYdpRhYnSDl-eoBiOBQc -O ./data/tr_types.csv

Downloading...
From: https://drive.google.com/uc?id=1EP3KrATWS1I_qYdpRhYnSDl-eoBiOBQc
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/tr_types.csv
100%|██████████████████████████████████████| 14.2k/14.2k [00:00<00:00, 19.4MB/s]


In [7]:
tr_types = pd.read_csv('./data/tr_types.csv', sep=';')
tr_types.head()

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...


In [8]:
types_contains_substrings = tr_types[(tr_types.tr_description.str.contains('POS')) | (tr_types.tr_description.str.contains('АТМ'))].tr_type

In [9]:
tr_types_samples.isin(types_contains_substrings).value_counts()[True] / len(tr_types_samples)

0.649

## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [10]:
top_transactions = transactions['tr_type'].value_counts(ascending=False, normalize=True).head(10)
top_transactions

1010    0.231117
2010    0.151166
7070    0.149006
1110    0.137658
1030    0.118975
2370    0.049830
7010    0.028972
7030    0.026078
7071    0.015781
1100    0.015476
Name: tr_type, dtype: float64

In [11]:
tr_types[tr_types.tr_type.isin(top_transactions.index)]

,tr_type,tr_description
39,7010,Взнос наличных через АТМ (в своем тер.банке)
47,7030,Перевод на карту (с карты) через АТМ (в предел...
59,7070,Перевод на карту (с карты) через Мобильный бан...
60,7071,Перевод на карту (с карты) через Мобильный бан...
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
106,2010,Выдача наличных в АТМ Сбербанк России
129,2370,Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [12]:
customers_amount = transactions[['customer_id', 'amount']]

In [13]:
income_best = (
    customers_amount[customers_amount.amount > 0].
    groupby('customer_id').
    sum().
    sort_values('amount', ascending=False).
    head(1)
)
income_best_customer = income_best.index.values.astype(int)[0]
income_best_customer

70780820

In [14]:
expense_best = (
    customers_amount[customers_amount.amount < 0].
    groupby('customer_id').
    sum().
    sort_values('amount').head(1)
)
expense_best_customer = expense_best.index.values.astype(int)[0]
expense_best_customer

70780820

In [15]:
abs(abs(income_best) - abs(expense_best)).amount.values.astype(float)[0]

1837317.980000019

    ## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [16]:
top_transactions_full = transactions[transactions.tr_type.isin(top_transactions.index)]

In [17]:
top_transactions_full.groupby('tr_type').amount.mean()

tr_type
1010    -19784.748640
1030     -5320.980222
1100    -44061.827262
1110    -32119.330371
2010   -136077.629325
2370   -205418.249032
7010    276391.789596
7030     86104.332909
7070     65569.831700
7071     66806.826623
Name: amount, dtype: float64

In [18]:
top_transactions_full.groupby('tr_type').amount.median()

tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64

In [19]:
customers_transactions = transactions[transactions.customer_id.isin([income_best_customer, expense_best_customer])]

In [20]:
customers_transactions.groupby('tr_type').amount.median()

tr_type
2010   -3368873.660
2330   -2245915.770
2370   -2245915.770
6110      10028.010
7020      10319.985
7021      28500.670
7030       8848.910
7031      12992.620
7034      14306.480
7040      14396.320
7070       8714.150
7071      12543.440
7074       9387.930
Name: amount, dtype: float64

In [21]:
customers_transactions.groupby('tr_type').amount.mean()

tr_type
2010   -2.941056e+06
2330   -2.382398e+06
2370   -2.218505e+06
6110    1.756293e+04
7020    1.465129e+04
7021    2.850067e+04
7030    1.406196e+04
7031    1.921036e+04
7034    1.430648e+04
7040    2.153833e+04
7070    1.384821e+04
7071    1.860057e+04
7074    2.281159e+04
Name: amount, dtype: float64

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [22]:
!gdown 10J8RzMIhoYHiad49r-oWNMAk-V5lo3OE -O ./data/tr_mcc_codes.csv

Downloading...
From: https://drive.google.com/uc?id=10J8RzMIhoYHiad49r-oWNMAk-V5lo3OE
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/tr_mcc_codes.csv
100%|██████████████████████████████████████| 14.9k/14.9k [00:00<00:00, 19.1MB/s]


In [23]:
!gdown 1FG1fopcmvMZ7GBaBOqQipccSeFoMUvNT -O ./data/gender_train.csv

Downloading...
From: https://drive.google.com/uc?id=1FG1fopcmvMZ7GBaBOqQipccSeFoMUvNT
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/gender_train.csv
100%|██████████████████████████████████████| 99.9k/99.9k [00:00<00:00, 3.74MB/s]


In [24]:
gender_train = pd.read_csv('./data/gender_train.csv')
tr_mcc_codes = pd.read_csv('./data/tr_mcc_codes.csv', sep=';')

merged_transactions = (
    pd.merge(transactions, gender_train, how='left').
    merge(tr_mcc_codes, how='inner').
    merge(tr_types, how='inner')
)
merged_transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [25]:
max_income = merged_transactions[merged_transactions['amount'] > 0].groupby(['gender', 'tr_type']).amount.sum()

In [26]:
zero_type_max_income = max_income[0].sort_values().head(10)
zero_type_max_income

tr_type
4051     1122.96
4210     2245.92
4110     2245.92
2370     2335.75
7075     6737.75
2110    22459.16
1010    31532.66
4100    40334.88
2210    68315.82
1210    83644.87
Name: amount, dtype: float64

In [27]:
first_type_max_income = max_income[1].sort_values().head(10)
first_type_max_income

tr_type
2020      2245.92
4110      4491.84
1010     35317.03
8100     64682.37
6000     79280.83
7041     87590.72
2370    113194.15
7015    121279.45
2110    179673.26
2010    224591.59
Name: amount, dtype: float64

In [28]:
pd.DataFrame(set(zero_type_max_income.index).intersection(first_type_max_income.index), columns=['tr_type', ])

,tr_type
0,2110
1,1010
2,2370
3,4110


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [29]:
merged_transactions.columns

Index(['customer_id', 'tr_datetime', 'mcc_code', 'tr_type', 'amount',
       'term_id', 'gender', 'mcc_description', 'tr_description'],
      dtype='object')

In [30]:
expenses_by_gender = merged_transactions[merged_transactions.amount <0 ].groupby(['gender', 'mcc_code']).amount.sum()
expenses_by_gender.head()

gender  mcc_code
0.0     742          -386207.68
        1711         -674897.69
        1799         -261626.73
        2741          -20746.42
        3000       -74431645.27
Name: amount, dtype: float64

In [31]:
expenses_by_gender.index.names

FrozenList(['gender', 'mcc_code'])

In [32]:
top_abs_expenses = abs(expenses_by_gender[0] - expenses_by_gender[1]).sort_values(ascending=False).head(10)
tr_mcc_codes[tr_mcc_codes.mcc_code.isin(top_abs_expenses.index)]

,mcc_code,mcc_description
22,4829,Денежные переводы
62,5511,"Легковой и грузовой транспорт — продажа, серви..."
64,5533,Автозапчасти и аксессуары
65,5541,Станции техобслуживания
75,5661,Обувные магазины
90,5812,"Места общественного питания, рестораны"
116,5977,Магазины косметики
123,6010,Финансовые институты — снятие наличности вручную
124,6011,Финансовые институты — снятие наличности автом...
162,7995,Транзакции по азартным играм


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [33]:
merged_transactions[['tr_day', 'tr_time']] = merged_transactions.tr_datetime.str.split(' ', expand=True)
merged_transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_day,tr_time
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,0,10:23:26
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,6,07:08:31
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8,07:06:10
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,08:49:03
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,14:12:08


In [34]:
merged_transactions[['tr_hour', 'tr_minute', 'tr_second']] = merged_transactions.tr_time.str.split(':', expand=True)
merged_transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_day,tr_time,tr_hour,tr_minute,tr_second
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,0,10:23:26,10,23,26
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,6,07:08:31,07,08,31
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8,07:06:10,07,06,10
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,08:49:03,08,49,03
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,14:12:08,14,12,08


In [38]:
merged_transactions = merged_transactions.drop(columns=['tr_datetime', 'tr_time'])
merged_transactions.head()

,customer_id,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_day,tr_hour,tr_minute,tr_second
0,39026145,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,0,10,23,26
1,39026145,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,6,07,08,31
2,39026145,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8,07,06,10
3,39026145,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,08,49,03
4,39026145,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,14,12,08


In [39]:
merged_transactions[(merged_transactions.amount < 0) & (merged_transactions.tr_hour.astype(int) <= 6)].groupby('gender').amount.count()

gender
0.0    43510
1.0    46694
Name: amount, dtype: int64